In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount='True')

from sklearn.decomposition import PCA, TruncatedSVD

Mounted at /content/gdrive


In [2]:
!pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
import glob
import librosa
import scipy.io.wavfile as wave
from scipy.fftpack import fft, ifft, fftshift
import matplotlib.pyplot as plt 
import numpy as np
from sklearn import mixture
import joblib

# ZFCC

In [4]:
def ZFCC ( signal, sr, n_zfcc, frame_length, hop_length) :

        signal = np.diff ( signal )
        signal = np.cumsum ( np.cumsum ( signal )  )
        kernel = np.ones ( frame_length ) / frame_length
        signal -=  np.convolve ( signal , kernel , mode = "same")
        zfcc = librosa.feature.mfcc(y=signal, sr=sr, hop_length=hop_length, n_mfcc=n_zfcc,win_length=int(0.05*sr))
        #print(zfcc.shape)
        return zfcc

# Training

In [41]:
def training(train_data_path,feat_train_path,trained_model_path, n_mix, max_it, n_mfcc, inc_mfcc,inc_mfccd, inc_mfccdd, inc_zfcc, feat_red_method):

  all_languages=glob.glob(train_data_path+'*')
  print(all_languages)

  directory=feat_train_path
  if not os.path.exists(directory):
      os.makedirs(directory)
      
  directory=trained_model_path
  if not os.path.exists(directory):
      os.makedirs(directory)

  feat_red = []

  for itr1 in range(0,len(all_languages)):

      wavs=glob.glob(all_languages[itr1]+'/*/*.wav')
      
      lang=(all_languages[itr1]).split("/")[-1]
      
      if not os.path.exists(directory):
          os.makedirs(directory)

      flag = [inc_mfcc, inc_mfccd, inc_mfccdd, inc_zfcc]
      count = 0
      for i in range(4):
        if flag[i] == True:
          count = count+1
      final_feat=np.empty([0, n_mfcc*count])
      for itr2 in range(0,len(wavs)):
          
          y, srr = librosa.load(wavs[itr2])
          y = librosa.resample(y, srr, sr)
          # sr=8000
          
          # hop_length=int(0.005*sr)
          mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=n_mfcc,win_length=int(0.05*sr))
          mfcc_delta = librosa.feature.delta(mfcc)
          mfcc_ddelta = librosa.feature.delta(mfcc_delta)
          zfcc = ZFCC(y, sr, n_mfcc, 2*hop_length, hop_length)

          if(mfcc.shape[1] != zfcc.shape[1]):
            zfcc = np.concatenate((zfcc,np.zeros((n_mfcc,1))),axis=1)

          if inc_zfcc == False and inc_mfcc == True and inc_mfccd == True and inc_mfccdd == True: #1
            feat = np.concatenate((mfcc, mfcc_delta, mfcc_ddelta),axis=0)
          elif inc_zfcc == False and inc_mfcc == True and inc_mfccd == True and inc_mfccdd == False: #2
            feat = np.concatenate((mfcc, mfcc_delta),axis=0)
          elif inc_zfcc == False and inc_mfcc == True and inc_mfccd == False and inc_mfccdd == True: #3
            feat = np.concatenate((mfcc, mfcc_ddelta),axis=0)
          elif inc_zfcc == False and inc_mfcc == False and inc_mfccd == True and inc_mfccdd == True: #4
            feat = np.concatenate((mfcc_delta, mfcc_ddelta),axis=0)
          elif inc_zfcc == False and inc_mfcc == True and inc_mfccd == False and inc_mfccdd == False: #5
            feat = mfcc
          elif inc_zfcc == False and inc_mfcc == False and inc_mfccd == False and inc_mfccdd == True: #6
            feat = mfcc_ddelta
          elif inc_zfcc == False and inc_mfcc == False and inc_mfccd == True and inc_mfccdd == False: #7
            feat = mfcc_delta
          elif inc_zfcc == True and inc_mfcc == True and inc_mfccd == True and inc_mfccdd == True: #8
            feat = np.concatenate((mfcc, mfcc_delta, mfcc_ddelta, zfcc),axis=0)
          elif inc_zfcc == True and inc_mfcc == True and inc_mfccd == True and inc_mfccdd == False: #9
            feat = np.concatenate((mfcc, mfcc_delta, zfcc),axis=0)
          elif inc_zfcc == True and inc_mfcc == True and inc_mfccd == False and inc_mfccdd == True: #10
            feat = np.concatenate((mfcc, mfcc_ddelta, zfcc),axis=0)
          elif inc_zfcc == True and inc_mfcc == False and inc_mfccd == True and inc_mfccdd == True: #11
            feat = np.concatenate((mfcc_delta, mfcc_ddelta, zfcc),axis=0)
          elif inc_zfcc == True and inc_mfcc == True and inc_mfccd == False and inc_mfccdd == False: #12
            feat = np.concatenate((mfcc, zfcc),axis=0)
          elif inc_zfcc == True and inc_mfcc == False and inc_mfccd == False and inc_mfccdd == True: #13
            feat = np.concatenate((mfcc_ddelta, zfcc),axis=0)
          elif inc_zfcc == True and inc_mfcc == False and inc_mfccd == True and inc_mfccdd == False: #14
            feat = np.concatenate((mfcc_delta, zfcc),axis=0)
          elif inc_zfcc == True and inc_mfcc == False and inc_mfccd == False and inc_mfccdd == False: #15
            feat = zfcc
          
          

          feat = feat.transpose()
          final_feat=np.concatenate((final_feat,feat),axis=0)
          
          

      if feat_red_method == "pca":
        feat_red.append(PCA(n_components = n_mfcc))
        feat_red[itr1].fit(final_feat)
        final_feat = feat_red[itr1].transform(final_feat)
      elif feat_red_method == "trunc_svd":
        feat_red.append(TruncatedSVD(n_components = n_mfcc-1))
        feat_red[itr1].fit(final_feat)
        final_feat = feat_red[itr1].transform(final_feat)

          #print(final_feat.shape)
      print(lang)    
      #np.savetxt(feat_train_path+lang+"_all_features.txt", final_feat, delimiter=",")

      try:
          #gmm = mixture.GaussianMixture(n_components=n_mixtures, covariance_type='diag' , max_iter = max_iterations ).fit(final_feat)
          gmm = mixture.GaussianMixture(n_components=n_mix, covariance_type='diag', max_iter = max_it, init_params = 'kmeans').fit(final_feat)
      except:
          print("ERROR : Error while training model for file "+lang)
          
      try:
          joblib.dump(gmm,trained_model_path+lang+'.pkl')
      except:
          print("ERROR : Error while saving model for "+lang)
          
  
  print("Training Completed")
  return feat_red

# Testing

In [40]:
def testing(test_data_path,feat_test,trained_model_path, n_mfcc, inc_mfcc, inc_mfccd, inc_mfccdd, inc_zfcc, feat_red, apply_feat_red):
    # train feature extraction
    all_languages=glob.glob(test_data_path+'*')

    import os
    directory=feat_test
    if not os.path.exists(directory):
        os.makedirs(directory)

    languages = { all_languages[k]:k for k in range(len(all_languages)) }

    num_test_cases={}
    tct={}
    for e in languages:
        num_test_cases[e.replace(test_data_path,'')]=len(os.listdir(e))-1
        tct[e.replace(test_data_path,'')]=0

    #print(num_test_cases)

    lang_names = { all_languages[k].replace(test_data_path,''):k for k in range(len(all_languages)) }

    total_languages=len(num_test_cases)

    confusion_matrix = np.zeros((total_languages,total_languages))

    for itr1 in range(0,len(all_languages)):
        
        wavs=glob.glob(all_languages[itr1]+'/*/*.wav')
        
        lang=(all_languages[itr1]).split("/")[-1]
        
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        flag = [inc_mfcc, inc_mfccd, inc_mfccdd, inc_zfcc]
        count = 0
        for i in range(4):
          if flag[i] == True:
            count = count+1
        final_feat=np.empty([0, n_mfcc*count])
        feat = np.empty([0,n_mfcc])
        for itr2 in range(0,len(wavs)):
            #print(wavs[itr2])
            
            y, srr = librosa.load(wavs[itr2])
            y = librosa.resample(y, srr, sr)
            # sr=8000

            mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=n_mfcc,win_length=int(0.05*sr))
            mfcc_delta = librosa.feature.delta(mfcc)
            mfcc_ddelta = librosa.feature.delta(mfcc_delta)
            zfcc = ZFCC(y, sr, n_mfcc, 2*hop_length, hop_length)

            if(mfcc.shape[1] != zfcc.shape[1]):
                zfcc = np.concatenate((zfcc,np.zeros((n_mfcc,1))),axis=1)

            if inc_zfcc == False and inc_mfcc == True and inc_mfccd == True and inc_mfccdd == True: #1
              feat = np.concatenate((mfcc, mfcc_delta, mfcc_ddelta),axis=0)
            elif inc_zfcc == False and inc_mfcc == True and inc_mfccd == True and inc_mfccdd == False: #2
              feat = np.concatenate((mfcc, mfcc_delta),axis=0)
            elif inc_zfcc == False and inc_mfcc == True and inc_mfccd == False and inc_mfccdd == True: #3
              feat = np.concatenate((mfcc, mfcc_ddelta),axis=0)
            elif inc_zfcc == False and inc_mfcc == False and inc_mfccd == True and inc_mfccdd == True: #4
              feat = np.concatenate((mfcc_delta, mfcc_ddelta),axis=0)
            elif inc_zfcc == False and inc_mfcc == True and inc_mfccd == False and inc_mfccdd == False: #5
              feat = mfcc
            elif inc_zfcc == False and inc_mfcc == False and inc_mfccd == False and inc_mfccdd == True: #6
              feat = mfcc_ddelta
            elif inc_zfcc == False and inc_mfcc == False and inc_mfccd == True and inc_mfccdd == False: #7
              feat = mfcc_delta
            elif inc_zfcc == True and inc_mfcc == True and inc_mfccd == True and inc_mfccdd == True: #8
              feat = np.concatenate((mfcc, mfcc_delta, mfcc_ddelta, zfcc),axis=0)
            elif inc_zfcc == True and inc_mfcc == True and inc_mfccd == True and inc_mfccdd == False: #9
              feat = np.concatenate((mfcc, mfcc_delta, zfcc),axis=0)
            elif inc_zfcc == True and inc_mfcc == True and inc_mfccd == False and inc_mfccdd == True: #10
              feat = np.concatenate((mfcc, mfcc_ddelta, zfcc),axis=0)
            elif inc_zfcc == True and inc_mfcc == False and inc_mfccd == True and inc_mfccdd == True: #11
              feat = np.concatenate((mfcc_delta, mfcc_ddelta, zfcc),axis=0)
            elif inc_zfcc == True and inc_mfcc == True and inc_mfccd == False and inc_mfccdd == False: #12
              feat = np.concatenate((mfcc, zfcc),axis=0)
            elif inc_zfcc == True and inc_mfcc == False and inc_mfccd == False and inc_mfccdd == True: #13
              feat = np.concatenate((mfcc_ddelta, zfcc),axis=0)
            elif inc_zfcc == True and inc_mfcc == False and inc_mfccd == True and inc_mfccdd == False: #14
              feat = np.concatenate((mfcc_delta, zfcc),axis=0)
            elif inc_zfcc == True and inc_mfcc == False and inc_mfccd == False and inc_mfccdd == False: #15
              feat = zfcc

            feat = feat.transpose()
            final_feat=np.concatenate((final_feat,feat),axis=0)

            if apply_feat_red == True:
              feat = feat_red[itr1].transform(feat)

            #print(final_feat.shape)
            max_score=-np.inf
            max_lang_name=""
            
            for modelfile in sorted(glob.glob(trained_model_path+'*.pkl')):
                gmm = joblib.load(modelfile) 
                score=gmm.score(feat)
                #print score
                if score>max_score:
                    max_score,max_lang_name=score,modelfile.replace(trained_model_path,'').replace('.pkl','')

            print(lang+" -> "+max_lang_name+(" Y" if lang==max_lang_name  else " N"))

            confusion_matrix[ lang_names[lang] ][lang_names[max_lang_name]]+=1

        tct[lang]+=len(wavs)

        #print(lang)
        #np.savetxt(feat_test+lang+"_all_features.txt", feat, delimiter=",")
        
    return tct,confusion_matrix,total_languages
  

# File Paths

In [29]:
# All paths should be changed according to your file locations

feat='/content/gdrive/MyDrive/clean_dataset/feat/'
feat_train='/content/gdrive/MyDrive/clean_dataset/train/'
feat_test='/content/gdrive/MyDrive/clean_dataset/test/'
trained_model='/content/gdrive/MyDrive/cleany_dataset/train_models/'
train_data='/content/gdrive/MyDrive/clean_dataset/traindata/'
test_data='/content/gdrive/MyDrive/clean_dataset/testdata/'

In [8]:
# for removing existing feature folders, models created
if os.path.exists(feat):
  !rm -rf feat
if os.path.exists(trained_model):
  !rm -rf trained_model
if os.path.exists(feat_train):
  !rm -rf feat_train

# Main

In [9]:

n_mixtures = 128
max_iterations = 100
calc_deltas=True
sr=8000
hop_length=int(0.005*sr)


In [ ]:
n_mixtures = 32
max_iterations = 50
n_mfcc = 13
calc_deltas=True
sr=8000
hop_length=int(0.005*sr)
inc_mfcc = False
inc_mfccd = False
inc_mfccdd = False
inc_zfcc = True
feat_red_method = ["none", "pca", "trunc_svd"]
feat_red = training(train_data,feat_train,trained_model, n_mixtures, max_iterations, n_mfcc, inc_mfcc, inc_mfccd, inc_mfccdd, inc_zfcc, "none")
tt,conf_mat,tot_spek = testing(test_data,feat_test,trained_model, n_mfcc, inc_mfcc, inc_mfccd, inc_mfccdd, inc_zfcc, feat_red, False)
print(tt)
#print("MFCC + MFCCD + MFCCDD")
print("Confusion Matrix:\n",conf_mat)
print("Accuracy: ",(sum([ conf_mat[i][j] if i==j  else 0 for i in range(tot_spek) for j in range(tot_spek) ] )*100)/float(sum([i for i in tt.values()])))

['/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/manipuri', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/gujarathi', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/telugu', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/marathi', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/odia', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/bengali', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/assamese']


In [43]:
n_mixtures = 16
max_iterations = 25
n_mfcc = 13
calc_deltas=True
sr=8000
hop_length=int(0.005*sr)
inc_mfcc = [False, True]
inc_mfccd = [False, True]
inc_mfccdd = [False, True]
inc_zfcc = [False, True]
feat_red_method = ["none", "pca", "trunc_svd"]

for i in range(2):
  for j in range(2):
    for k in range(2):
      for l in range(2):
        if (i != 0 or j!= 0 or k!=0 or l != 0):
          #feat_red = []
          feat_red = training(train_data,feat_train,trained_model, n_mixtures, max_iterations, n_mfcc, inc_mfcc[i], inc_mfccd[j], inc_mfccdd[k], inc_zfcc[l], "none")
          tt,conf_mat,tot_spek = testing(test_data,feat_test,trained_model, n_mfcc, inc_mfcc[i], inc_mfccd[j], inc_mfccdd[k], inc_zfcc[l], feat_red, False)
          print(tt)
          print("MFCC" + str(inc_mfcc[i]))
          print("MFCCD" + str(inc_mfccd[j]))
          print("MFCCDD" + str(inc_mfccdd[k]))
          print("ZFCC" + str(inc_zfcc[l]))
          print("Confusion Matrix:\n",conf_mat)
          print("Accuracy: ",(sum([ conf_mat[i][j] if i==j  else 0 for i in range(tot_spek) for j in range(tot_spek) ] )*100)/float(sum([i for i in tt.values()])))

['/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/manipuri', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/gujarathi', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/telugu', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/marathi', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/odia', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/bengali', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/assamese']
MFCCFalse
MFCCDFalse
MFCCDDFalse
ZFCCTrue
Confusion Matrix:
 [[ 0.  0. 13.  4.  0.  3.  0.]
 [ 0.  8.  4.  5.  0.  0.  3.]
 [ 0.  1.  6. 10.  2.  1.  0.]
 [ 0.  0.  0. 19.  1.  0.  0.]
 [ 0.  2.  4.  7.  6.  1.  0.]
 [ 0.  0.  6.  7.  1.  6.  0.]
 [ 2.  0.  5.  5.  1.  3.  4.]]
Accuracy:  35.0
['/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/manipuri', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/gujarathi', '/content/gdrive/MyDrive/enhanced_factory_dataset/traindata/telugu', '

KeyboardInterrupt: ignored